# Iterative AIVC models building (if using github - add credentials in cell 2)
(This summary is out of sync - to be edited)
### Implemented pipeline:

#### 1.- Data gathering
#### 2.- Classify data (probabilistic classification)
#### 3.- Sample data from each strate of classifications (high confidence, medium, low)
#### 4.- Manually validate data
#### 5.- Append data to training files
#### 6.- Retrain new models
#### 7.- Gather new data

In [1]:
import pandas as pd
import numpy as np
import os
import pickle
import re
import requests as req
import json

## Deep Learning imports for the classifiers
os.environ['KERAS_BACKEND']='theano'

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, Concatenate
from keras.models import Model


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.utils.multiclass import unique_labels
from sklearn import metrics
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer

from urllib.request import urlopen
import cloudpickle as cp
import urllib
import urllib.request 
import cloudstorage
from github import Github
from github import InputGitTreeElement


Using Theano backend.


In [2]:
###*******START-POPULATE GIT_HUB Credentials ********###
#To read data files directly from  github - usr/pssword not necessary for read
USE_GIT_HUB = False 

#To read pickle files from github - usr/pssword not necessary for read
READ_PICKLE_FROM_GIT_HUB = False  
#leave READ_PICKLE_FROM_GIT_HUB set to False as pickle files too big for github if not zipped(to discuss  with David)



#To push data files directly to  github - usr/pssword required
GIT_USER = '***' 
GIT_PSWD = '***'
GIT_REPO = 'AIVC'
###*******END-POPULATE GIT_HUB Credentials ********###


#Classificaiton NETWORKs Configuration parameters
MAX_SEQUENCE_LENGTH = 32
MAX_NB_WORDS = 10000
EMBEDDING_DIM = 100 ## 100, 200 or 300
VALIDATION_SPLIT = 0.2
NUM_EPOCHS = 20

This notebook will read the data files directly from github: the following is the hirearchy.

- BASE_DIR
     - Gathered_data
         - Conventions
         - Software_characteristics
     - Classification_results
         - Conventions
         - Software_characteristics
     - Training_data
         - Conventions
         - Software_characteristics

In [3]:
RESULTS_SAMPLING_PERCENTAJE = 30

# Data gathering

To be done for different sources, to have high variability. It's important to keep tracking of where does each sentence come from (add a label of provenance).
Identified data sources:
    - Google
    - Github
    - Semantic Scholar
    
    


# Helper functions

In [15]:
import  base64

def remove_excluded_files(file_list):
    cleaned_file_list = []
    for f in file_list:
        #and "." in f to exclude directories from the list.
        if not f.startswith( '.' ) and not "random" in f and "gathered_" in f and not f.startswith("_") and "." in f:
            cleaned_file_list.append(f)
    return cleaned_file_list
        
        
def rreplace(s, old, new, occurrence):
    li = s.rsplit(old, occurrence)
    return new.join(li)


def clean_file_name(name, replacements2=[]):
    
    replacements=[".txt", ".csv", ".tsv"]
    
    for r in replacements:
        name = name.replace(r, "")
        
    for r in replacements2:
        name = name.replace(r, "")
    return name


def save_to_github(git_user, git_password, git_repo, my_file_list, push_to_git_as):
    '''
    in order to push a file to github it must first be stored locally, then pushed
    this local location can also be local to a virtual machine. 
    takes: 
            git username, password, repo, 
            a list of files to push to git ie the full local location of file,
            a matching list of paths to push each file to in Git hub 
    '''
    display(git_user, git_password, git_repo, my_file_list, push_to_git_as)
    user = git_user
    password = git_password
    url = git_repo
    file_list = []  #push these list of files to git
    file_names = [] #push to this location in git
    
    
    try:
        g = Github(user, password)
        repo = g.get_user().get_repo(url)

        file_list = my_file_list 
        file_names = push_to_git_as
                 
        commit_message = 'training data updated via the audit tool'
    
        master_ref = repo.get_git_ref('heads/master')
        master_sha = master_ref.object.sha
        base_tree = repo.get_git_tree(master_sha)
        element_list = list()

        for i, entry in enumerate(file_list):
            print ("file to commit:", entry)
            
            with open(entry) as input_file:
                #data = input_file.read()   #works with non zip file
                data = base64.b64encode(open(entry, "rb").read())
                
            if entry.endswith('.png'):
                data = base64.b64encode(data) 
                

            blob = repo.create_git_blob(data.decode("utf-8"), "base64")
            element = InputGitTreeElement(path=file_names[i], mode='100644', type='blob', sha=blob.sha)

            #push to git as file_names[i]
            print ("push to git as:", file_names[i])
            
            #element = InputGitTreeElement(file_names[i], '100644', 'blob', data)
            
            #element_list is a list of InputGitTreeElement. 
            #Each one corresponds to a file. 
            # the 'content' of InputGitTreeElement can only be of type 'str' or 'unicode'. 
            #When I load a file to memory I have type 'bytes'. 
            #What is the right way to encode those bytes to str or unicode to upload a .zip
            element_list.append(element)   
        
        tree = repo.create_git_tree(element_list, base_tree)
        print("got here 2")
        parent = repo.get_git_commit(master_sha)
        print("got here 3")
        commit = repo.create_git_commit(commit_message, tree, [parent])
        master_ref.edit(commit.sha)
        print ("File commited to github :", commit)
    except e:
        print("")
        print ("GITHUB SUBMIT FAILED:")
        print ("Are your github login credentials correct?")
        print ("Are you a collaberator in the repo?")
        print(e)


# Read conventions data and  software characteristics data
either from local file system for local testing or from github to allow full synchronization across collaborators.

In [5]:
gathered_conventions_files = []
gathered_characteristics_files = []
gathered_conventions_data = {}
gathered_software_characteristics_data = {}

In [18]:
if(USE_GIT_HUB == False):
    
    MODELS_DIR = "./Data/Iterative-models-building/Models/"
    BASE_DIR = "./Data/Iterative-models-building"
    
    
    GATHERED_DATA_FOLDER = os.path.join(BASE_DIR, "Gathered_data")
    CONVS_DIR_NAME = "Conventions"
    SOFT_CHARS_DIR_NAME = "Software_characteristics"
    GATHERED_SOFTWARE_CHARS_DIR = os.path.join(GATHERED_DATA_FOLDER, SOFT_CHARS_DIR_NAME)
    GATHERED_CONVENTIONS_DIR = os.path.join(GATHERED_DATA_FOLDER, CONVS_DIR_NAME)
    
    
    #For each file() in gathered conventions folder
    gathered_conventions_files = [f for f in os.listdir(GATHERED_CONVENTIONS_DIR) ]
    gathered_conventions_files = remove_excluded_files(gathered_conventions_files) 
    for f in gathered_conventions_files:
        gathered_conventions_data[clean_file_name(f)] = pd.read_csv(os.path.join(GATHERED_CONVENTIONS_DIR, f), sep='\t')
      

    #For each file in gathered software characteristics folder
    gathered_characteristics_files = [f for f in os.listdir(GATHERED_SOFTWARE_CHARS_DIR) ]
    gathered_characteristics_files = remove_excluded_files(gathered_characteristics_files)
    for f in gathered_characteristics_files:
        gathered_software_characteristics_data[clean_file_name(f)] = pd.read_csv(os.path.join(GATHERED_SOFTWARE_CHARS_DIR, f), sep='\t')
    

In [20]:
# the entire cell is only valid if using github as data source
if(USE_GIT_HUB == True): 
    
    # read data files for source data directly from github.
    # to obtain the id for the folder, navigate the tree using
    # https://api.github.com/repos/{owner}/{repo}/git/trees/master
    # e.g https://api.github.com/repos/aideenf/AIVC/git/trees/master
    # once navigated each directory will be of format
    # https://api.github.com/repos/aideenf/AIVC/git/trees/{dir_ref}
    
    

    ####need an alternative for these
    MODELS_DIR = "./Data/Iterative-models-building/Models/"
  
    
    BASE_DIR = "./Data/Iterative-models-building"
    CONVS_DIR_NAME = "Conventions"
    SOFT_CHARS_DIR_NAME = "Software_characteristics"
    ####
     
    GATHERED_CONVENTIONS_DIR = "https://api.github.com/repos/aideenf/AIVC/git/trees/048349b4dd81d95a17129e7fcd5418bdca8309b3"
    resp = req.get(GATHERED_CONVENTIONS_DIR)
    response  = json.loads(resp.text)
    for value in response['tree']:
        gathered_conventions_files.append(value['path'])
    gathered_conventions_files = remove_excluded_files(gathered_conventions_files)
    
    
    GATHERED_SOFTWARE_CHARS_DIR = "https://api.github.com/repos/aideenf/AIVC/git/trees/149a2b2f9696e575a4106e85002afda36a86e5fe"
    resp = req.get(GATHERED_SOFTWARE_CHARS_DIR)
    response  = json.loads(resp.text)
    for value in response['tree']:
        gathered_characteristics_files.append(value['path'])
    gathered_characteristics_files = remove_excluded_files(gathered_characteristics_files)
    
    #Read conventions data file from github. 
    for f in gathered_conventions_files:
        urlBase  = 'https://raw.githubusercontent.com/aideenf/AIVC/master/cp_wssc/Data/Iterative-models-building/Gathered_data/Conventions/'
        gathered_conventions_data[clean_file_name(f)] = pd.read_csv(urlBase+f, sep='\t', error_bad_lines=False)
        # NOTE: "truncated": false  we should check for truncated = true to do follow on call to get all files
        
        
    #Read software characteristics data file from github. 
    for f in gathered_characteristics_files:
        urlBase  = 'https://raw.githubusercontent.com/aideenf/AIVC/master/cp_wssc/Data/Iterative-models-building/Gathered_data/Software_characteristics/'
        gathered_software_characteristics_data[clean_file_name(f)] = pd.read_csv(urlBase+f, sep='\t', error_bad_lines=False)

In [21]:
display ("Now we have a dictionary(gathered_conventions_data) of dataframes one entry for each gathered sentence source:" , gathered_conventions_files)
display ("Now we have a dictionary(gathered_software_characteristics_data) of dataframes one entry for each SW character source:" , gathered_characteristics_files)

display (gathered_conventions_data[list(gathered_conventions_data.keys())[0]].head(1))
display (gathered_software_characteristics_data[list(gathered_software_characteristics_data.keys())[0]].head(1))


'Now we have a dictionary(gathered_conventions_data) of dataframes one entry for each gathered sentence source:'

['gathered_news_sentences.tsv']

'Now we have a dictionary(gathered_software_characteristics_data) of dataframes one entry for each SW character source:'

['gathered_licensing.tsv',
 'gathered_advantages.tsv',
 'gathered_contributions.tsv',
 'gathered_efficiency.tsv',
 'gathered_portability.tsv',
 'gathered_functionalities.tsv']

,sentence,provenance
0,Monsanto And Bayer Are Set To Merge. Here's Wh...,HuffPost_News


,sentence,provenance
0,MIT License Copyright (c) 2018 Oleksii Trekhl...,licensing


# Clasification - load the classifiers from pickle files

Classify, using each of the classifiers, the gathered sentences

In [22]:
## Read pickled classifiers
# Note: pickle is used for serializing and de-serializing a Python object structure. 
# Any object in python can be pickled so that it can be saved on disk. 
# What pickle does is that it “serialises” the object first before writing it to file. 
# Pickling is a way to convert a python object (list, dict, etc.) into a character stream



## read convention models from pickle file from gitHub or local 
if(READ_PICKLE_FROM_GIT_HUB == True): 
    
    CONV_MODEL_PCKL_URL = 'https://raw.githubusercontent.com/aideenf/AIVC/master/cp_wssc/Data/Iterative-models-building/Models/conv_models_items.pickle'
    CHARACT_MODEL_PCKL_URL = 'https://github.com/aideenf/AIVC/blob/master/cp_wssc/Data/Iterative-models-building/Models/charact_models_items.pickle'
    
    #make sure updated pickle files are in github
    convention_convnet_items = pickle.load(urllib.request.urlopen(CONV_MODEL_PCKL_URL))
    characteristics_convnet_items = pickle.load(urllib.request.urlopen(CHARACT_MODEL_PCKL_URL))
    
    
# or local
elif(READ_PICKLE_FROM_GIT_HUB == False):
    
    f = open( os.path.join(MODELS_DIR, 'conv_models_items.pickle'), 'rb') 
    convention_convnet_items = pickle.load(f)
    f.close()
    f = open( os.path.join(MODELS_DIR, 'charact_models_items.pickle'), 'rb')
    characteristics_convnet_items = pickle.load(f)
    f.close()
    


## Load the  convention models from the downloaded pickle file             
_conventions_models = convention_convnet_items['model'] 
_conventions_tokenizers = convention_convnet_items['tokenizer'] 
_conventions_data_val_x = convention_convnet_items['_x_val'] 
_conventions_data_val_y = convention_convnet_items['_y_val'] 
_conventions_train_histories = convention_convnet_items['train_history'] 
 

## Load the convention models from the downloaded pickle file
_characteristics_models = characteristics_convnet_items['model'] 
_characteristics_tokenizers = characteristics_convnet_items['tokenizer'] 
_characteristics_data_val_x = characteristics_convnet_items['_x_val'] 
_characteristics_data_val_y = characteristics_convnet_items['_y_val'] 
_characteristics_train_histories = characteristics_convnet_items['train_history'] 



In [23]:
def get_model_matches_proba(sequences, model):
    data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
    preds = model.predict(data)

    return preds[:,1]


def calculate_matches(sentences, _models, _tokenizers):
    _repos_matches = {}

    ## Getting classification confidence per model for each repo
    for model_key in _models.keys():
            
        tokenized_sentences = _tokenizers[model_key].texts_to_sequences(sentences)
        preds = get_model_matches_proba(tokenized_sentences, _models[model_key])

        _repos_matches[clean_file_name(model_key)] = preds
    return _repos_matches

In [24]:
## TODO: Put here all sentences from gathered data 
#data_sentences = np.array(["This would be talking about efficiency in industrial terms", "just one random example", "please, classify this a green world"])
#data_sentences_provenance = np.array(["Google", "Github", "Semantic Scholar"])

#AF: Check with david what the above comments refer to

data_sentences = []
data_sentences_provenance = []

#for each data source in the dataframe dictionary
for k in gathered_conventions_data.keys():
    #retrieve the dataFrame for the convention
    conv_data_df = gathered_conventions_data[k]
    #aggregate the source
    provenances = conv_data_df['provenance'].values
    #aggregate the sentence
    texts = conv_data_df['sentence'].values
    for i in range(len(conv_data_df)):
        data_sentences.append( texts[i])
        data_sentences_provenance.append(provenances[i])

#Now we have a list of sentences and sentence sources which we will 
#input to the trained model and later we will evaluate and audit 
print ("Example sentence/source from combination of all data sources:")
print ("*Sentence:", data_sentences[0])
print ("*Source:", data_sentences_provenance[0])

Example sentence/source from combination of all data sources:
*Sentence: Monsanto And Bayer Are Set To Merge. Here's Why You Should Care.“Together they will influence markets all over the world on a scale we’ve never seen before.”
*Source: HuffPost_News


# Pass all the combined convention sentences through each convention classifier. 

In [25]:
conventions_classifications = calculate_matches( data_sentences, _conventions_models, _conventions_tokenizers)
display (conventions_classifications)

{'training_Domestic': array([0.02122647, 0.1334793 , 0.01104365, ..., 0.8555667 , 0.79177034,
        0.03241986], dtype=float32),
 'training_Civic': array([0.01864899, 0.00151586, 0.01796461, ..., 0.00085881, 0.01726504,
        0.10831312], dtype=float32),
 'training_Project': array([0.19300894, 0.02249742, 0.01060525, ..., 0.70351034, 0.2050803 ,
        0.32795954], dtype=float32),
 'training_Inspired': array([2.0972365e-01, 1.6293422e-04, 2.5358060e-03, ..., 4.8711628e-02,
        7.7970433e-01, 7.3070283e-04], dtype=float32),
 'training_Green': array([0.01180633, 0.02937861, 0.00741292, ..., 0.00697178, 0.42741475,
        0.5046879 ], dtype=float32),
 'training_Market': array([6.0658250e-04, 1.0000000e+00, 9.9986148e-01, ..., 5.1253510e-04,
        4.5718235e-04, 2.7296541e-05], dtype=float32),
 'training_Industrial': array([4.2383480e-03, 3.4371912e-09, 1.5511032e-03, ..., 7.3838014e-06,
        7.6878205e-05, 4.2075150e-08], dtype=float32),
 'training_Renown': array([0.5349886

# We will pass the combined software characteristic sentences through each characteristic classifier.

In [26]:
characteristics_classifications = calculate_matches( data_sentences, _characteristics_models, _characteristics_tokenizers)
display (characteristics_classifications)

{'training_advantages': array([0.25682357, 0.76273876, 0.6990553 , ..., 0.78577375, 0.24470451,
        0.9402202 ], dtype=float32),
 'training_usability': array([0.11924803, 0.01303323, 0.04906897, ..., 0.7829037 , 0.63326895,
        0.44374424], dtype=float32),
 'training_contributions': array([0.7201761 , 0.00114743, 0.9980919 , ..., 0.47499472, 0.01840728,
        0.3757422 ], dtype=float32),
 'training_efficiency': array([1.0176024e-02, 5.4021883e-01, 9.9776417e-01, ..., 3.4234970e-06,
        1.4901470e-01, 9.8053106e-06], dtype=float32),
 'training_licensing': array([2.7428430e-06, 1.2626502e-04, 3.4481297e-07, ..., 1.0169871e-05,
        2.6958523e-07, 3.1210273e-04], dtype=float32),
 'training_reliability:maintanability': array([0.20985408, 0.26234806, 0.25060442, ..., 0.40264472, 0.12399551,
        0.18416   ], dtype=float32),
 'training_functionalities': array([2.1786450e-03, 1.6658570e-01, 9.1681159e-06, ..., 7.6224906e-03,
        1.2401952e-03, 3.0047188e-03], dtype=flo

# Annotate data for each type of classifications (high confidence, medium, low + the confidence score)

In [27]:
class ClassificationResult:
        
    def __init__(self, text, value, level, provenance='Unknown'):
        self.text = text
        self.confidence_value = value
        self.confidence_level = level
        self.data_provenance = provenance
        
    def __str__(self):
        return "{} -- {} -- {}".format(self.text, self.confidence_value, self.confidence_level)
    
    def __repr__(self):
        return "{} -- {} -- {}".format(self.text, self.confidence_value, self.confidence_level)
        
        
def split_sentences_by_confidence (calculated_classifications, _sentences, _sentences_provenance):

    stratified_classifications = {}

    for k in calculated_classifications.keys():
        classifications = calculated_classifications[k]

        ## Low level percentile
        low_percentile = np.percentile(classifications, 33)#0.2
        classifications_low = np.where(classifications<=low_percentile)[0]
        #print(classifications_low)

        ## Medium level percentile
        medium_percentile = np.percentile(classifications, 66)#0.8
        classifications_medium = np.where((classifications<=medium_percentile) & (classifications>low_percentile))[0]
        #print(classifications_medium)

        ## High level percentile
        top_percentile = np.percentile(classifications, 100) #1.0
        classifications_top = np.where((classifications<=top_percentile) & (classifications>medium_percentile))[0]
        #print(classifications_top)
       
        classified_sentences = []
        #print(classifications_low)
        for i1 in classifications_low:
            c1 = ClassificationResult(_sentences[i1], classifications[i1], "Low", _sentences_provenance[i1])
            classified_sentences.append(c1)

        for i2 in classifications_medium:
            c2 = ClassificationResult(_sentences[i2], classifications[i2], "Medium", _sentences_provenance[i2])
            classified_sentences.append(c2)

        for i3 in classifications_top:
            c3 = ClassificationResult(_sentences[i3], classifications[i3], "High", _sentences_provenance[i3])
            classified_sentences.append(c3)
            
        stratified_classifications[clean_file_name(k)] = classified_sentences
    return stratified_classifications

# Conventions results sampling

In [29]:
#AF - Modified below code (line 13 and 17) to add the "convention" in question to the generated file.

conv_stratified_classifications = split_sentences_by_confidence(conventions_classifications, data_sentences, data_sentences_provenance)

file_list = []
  
for k in conv_stratified_classifications.keys():
    print (k)
    with open(os.path.join(BASE_DIR, "Classification results", CONVS_DIR_NAME, "{}_stratified_classifications.tsv".format(k)), "w")as f3:
        
        file_list.append(BASE_DIR +"/Classification results/"+CONVS_DIR_NAME + "/{}_stratified_classifications.tsv".format(k))
        
        f3.write("{}\t{}\t{}\t{}\t{}\n".format("text", "convention", "confidence_value", "confidence_level", "data_provenance"))
    
        for c in conv_stratified_classifications[k]:
            convention = k.replace("training_", "").lower()     
            f3.write("{}\t{}\t{}\t{}\t{}\n".format(c.text, convention, c.confidence_value, c.confidence_level, c.data_provenance))
        f3.close()
        

        
##############CODE ADDED BY AIDEEN#######################################################
#Create an aggregated DataFrame and then a tsv/csv file of the full data for auditing
#This code is to create a file and push it to git hub to be read by the audit tool only
# This is independant piece of code, removing it will only remove this functionality
# leaving it here means that when we retrain the model this file will be auto updated with
# new labels. (to discuss with David - to revert this to the format that can be used by the 
# model training process)
df_list = [pd.read_csv(file, sep='\t') for file in file_list]
#concatenate them 
print("Concatenate all files to one")
convention_sentences_df = pd.concat(df_list)
## We will perform one hot encoding
print("Applying one hot encoding to convention to have format suitable for audit tool (binary and for all)")
convention_sentences_df["convention"] = pd.Categorical(convention_sentences_df["convention"])
dfDummies = pd.get_dummies(convention_sentences_df["convention"], prefix="convention")
convention_sentences_df = pd.concat([convention_sentences_df, dfDummies], axis=1)
convention_sentences_df = convention_sentences_df.drop(["convention"], axis=1)

## Now we will save a file 'audit_training_data.tsv'  and zipped version locally which contains all of the results
file_name = 'audit_training_data.tsv'
file_name_zip = 'audit_training_data.gz'

file_path = BASE_DIR +"/Classification results/"+CONVS_DIR_NAME +"/"
convention_sentences_df.to_csv(file_path + file_name)
convention_sentences_df.to_csv(file_path + file_name_zip, compression='gzip')

## We will also push this to git hub for use by audit tool
if(USE_GIT_HUB == True):
    #Call helper file to zip the file for push to github
    my_file_list = [file_path + file_name_zip]
    push_to_git_as = ['cp_wssc/Data/Iterative-models-building/Classification results/Conventions/'+file_name_zip]
    save_to_github(GIT_USER, GIT_PSWD, GIT_REPO, my_file_list, push_to_git_as)
    
display (convention_sentences_df.head(2))
display (convention_sentences_df.tail(2))

############################END CODE ADDED BY AIDEEN####################################################
    

training_Domestic
training_Civic
training_Project
training_Inspired
training_Green
training_Market
training_Industrial
training_Renown
Concatenate all files to one
Applying one hot encoding to convention to have format suitable for audit tool (binary and for all)


,text,confidence_value,confidence_level,data_provenance,convention_civic,convention_domestic,convention_green,convention_industrial,convention_inspired,convention_market,convention_project,convention_renown
0,Monsanto And Bayer Are Set To Merge. Here's Wh...,0.021226,Low,HuffPost_News,0,1,0,0,0,0,0,0
1,Obamacare Premiums Will Be Way Higher Next Yea...,0.011044,Low,HuffPost_News,0,1,0,0,0,0,0,0


,text,confidence_value,confidence_level,data_provenance,convention_civic,convention_domestic,convention_green,convention_industrial,convention_inspired,convention_market,convention_project,convention_renown
2683,Fictional And Real Life Women Kick Butt In The...,0.678236,High,HuffPost_News,0,0,0,0,0,0,0,1
2684,Green Activists: 50 And OlderIf you look at so...,0.939327,High,HuffPost_News,0,0,0,0,0,0,0,1


In [ ]:
print(conv_stratified_classifications['training_Domestic'][0].text)
print(conv_stratified_classifications['training_Domestic'][1].text)

# software Characteristics results sampling -  

    - Nothing modified from below here.

    - AF: As per Meeting the focus should be on the conventions and labeling convention data

In [ ]:

chars_stratified_classifications = split_sentences_by_confidence(characteristics_classifications, data_sentences, data_sentences_provenance)
for k in conv_stratified_classifications.keys():
    with open(os.path.join(BASE_DIR, "Classification results", SOFT_CHARS_DIR_NAME, "{}_stratified_classifications.tsv".format(k)), "w")as f3:

        f3.write("{}\t{}\t{}\n".format("text", "confidence_value", "confidence_level", "data_provenance"))
    
        for c in conv_stratified_classifications[k]:
            
            f3.write("{}\t{}\t{}\t{}\n".format(c.text, c.confidence_value, c.confidence_level, c.data_provenance))

        f3.close()


## Next steps to be done manually:

### ~~4.- Manually validate data~~
### ~~5.- Append data to training files ~~



## 6.- Retrain new models

In [ ]:
def train_new_text_pipelineNB(texts, labels1):
    text_clf = Pipeline([
        ('vect', CountVectorizer(max_df=0.85, stop_words=stopwords, max_features=FEATURES_LENGTH)),
        ('tfidf', TfidfTransformer(smooth_idf=True,use_idf=True)),
        ('clf', MultinomialNB()),
    ])
    
    text_clf.fit(texts, labels1)  
    
    return text_clf

In [ ]:
mypath = "Data/VALIDATED_DATA/Conventions/"
training_files = [f for f in os.listdir(mypath) if (os.path.isfile(os.path.join(mypath, f)) and "training" in f and not f.startswith( '.' )) and not "random" in f]#[:2]
training_files

In [ ]:
FEATURES_LENGTH = 3000
VALIDATION_SPLIT = 0.25
GENERATE_NEW_TRAINING_FILES = False

In [ ]:

def get_stop_words(stop_file_path):
    """load stop words """
    
    with open(stop_file_path, 'r', encoding="utf-8") as f:
        stopwords = f.readlines()
        stop_set = set(m.strip() for m in stopwords)
        return frozenset(stop_set)
#load a set of stop words
stopwords=get_stop_words("resources/stopwords.txt")

In [ ]:
def clean_line(X):
    
    stemmer = WordNetLemmatizer()

    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(X))

    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 

    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)

    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)

    # Converting to Lowercase
    document = document.lower()

    # Lemmatization
    document = document.split()

    document2 = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document2)

    return document

In [ ]:
def clean_str(text):
    documents = []
    ret = documents
    if type(text == "list"):
        #print("list: ", text)
        for X in text:
            documents.append(clean_line(X))
    
        ret =""
        if(len(documents)>1):

            for d in documents:
                ret+=d+"\n"
        else:
            ret = documents[0]
        
    elif type(text)=="str":
        p#rint("str")
        ret = clear_line(text)
    return ret

In [ ]:
_conv_ML_models = {}
_test_data_x = {}
_test_data_y = {}


for f in training_files:
    print("----------------------------------------------------------------")
    print("            {}                  ".format(f))
    print("----------------------------------------------------------------")

    
    
    ## USING licensing text from github
    data_train = pd.read_csv(mypath+f, sep='\t')

    #data_train = pd.read_csv('Data/TRAINING_0/training_contributions.txt', sep='\t')

    print(data_train.shape)

    _texts = []
    _labels = []

    for idx in range(data_train.text.shape[0]):
        
        text = [data_train.text[idx]] #BeautifulSoup(data_train.text[idx])
        
        _texts.append(clean_str(text))#.encode('ascii','ignore')))
        _labels.append(int(data_train.category[idx]))
        
    #SHUFFLE and DATA SPLITTING 
    x_train, x_val, y_train, y_val = train_test_split(_texts, _labels, test_size=VALIDATION_SPLIT, random_state=42)
    
    _test_data_x[f] = np.array(x_val)[np.where(np.array(y_val)==1)]
    _test_data_y[f] = np.array(y_val)[np.where(np.array(y_val)==1)]
    
    model1 = train_new_text_pipelineNB(x_train, y_train)
    tmp_pred = model1.predict(x_val)
    print("Accuracy: {}".format(np.mean(tmp_pred == y_val)))

    _conv_ML_models[f] = model1
    
    #print("  Naive-Bayes: ",mean1)
    #print("  Random-Forest: ",mean2)
    #print(metrics.classification_report(_labels, pred))
        
    print("\n\n\n")


In [ ]:
def get_model_matches_proba_ML(_data, model):
    preds = model.predict_proba(_data)

    return preds[:,1]

def calculate_matches_ML(_sentences, _models):
    _repos_matches = {}

    ## Getting classification confidence per model for each repo
    for model_key in _models.keys():
            

        preds = get_model_matches_proba_ML(_sentences, _models[model_key])

        _repos_matches[clean_file_name(model_key)] = preds
    return _repos_matches

In [ ]:
def calculate_matches_mixture(_sentences, _modelsML, _modelsDL, _tokenizersDL):
    _repos_matches = {}

    ## Getting classification confidence per model for each repo
    for model_key in _modelsML.keys():
            
        tokenized_sentences = _tokenizersDL[model_key].texts_to_sequences(_sentences)
        preds1 = get_model_matches_proba(tokenized_sentences, _modelsDL[model_key])
        preds2 = get_model_matches_proba_ML(_sentences, _modelsML[model_key])

        _repos_matches[clean_file_name(model_key)] = preds1+preds2
    return _repos_matches
    

In [ ]:
#conv_ML_matches = calculate_matches_ML( data_sentences, _conv_ML_models)
conv_ML_matches = calculate_matches_mixture( data_sentences, _conv_ML_models, _conventions_models, _conventions_tokenizers)

In [ ]:
_conv_ML_models['training_Domestic.txt'].predict_proba([data_sentences[0]])

In [ ]:
def get_confidence_value(C):
    return C.confidence_value

In [ ]:
## Conventions results sampling
conv_stratified_classifications = split_sentences_by_confidence(conv_ML_matches, data_sentences, data_sentences_provenance)

for k in conv_stratified_classifications.keys():
    with open(os.path.join(BASE_DIR, "Classification results", CONVS_DIR_NAME, "ML_{}_stratified_classifications.tsv".format(k)), "w")as f3:

        f3.write("{}\t{}\t{}\t{}\n".format("text", "confidence_value", "confidence_level", "data_provenance"))
    
        for c in sorted(conv_stratified_classifications[k],key=get_confidence_value, reverse=True):
            
            f3.write("{}\t{}\t{}\t{}\n".format(c.text, c.confidence_value, c.confidence_level, c.data_provenance))

        f3.close()
    

In [ ]:
conv_stratified_classifications.keys()

In [ ]:
conv_stratified_classifications['training_Civic'],